In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook as tqdm
import pickle

In [2]:
links = []
for page_num in tqdm(range(806)):
    response = requests.get(f"https://www.magyarhirlap.hu/kereses?tol=2019-01-01&ig=2019-12-31&page={page_num + 1}")
    soup = BeautifulSoup(response.content, 'html.parser')
    titles = soup.select("h2")
    links.extend(["https://www.magyarhirlap.hu/" + title.find("a").get("href") for title in titles])
with open('links.p', 'wb') as f:
    pickle.dump(links, f)

In [30]:
magyar_hirlap = {}
for link in tqdm(links):
    response = requests.get(link)
    if response.status_code == 200:
        related_urls = []
        soup = BeautifulSoup(response.content, 'html.parser')
        try:
            datetime = soup.find("small", {"class": "time"}).text.replace("\n", "")
        except:
            datetime = None
        try:
            author = soup.find("small", {"class": "author"}).text.replace("\n", "").replace("\t", "")
            if "MTI" in author:
                related_urls.append("MTI")
        except:
            pass
        paragraphs = soup.find("div", {"class": "text-holder"}).find_all("p")
        for paragraph in paragraphs:
            if "MTI" in paragraph.text or "mti" in paragraph.text or "Mti" in paragraph.text:
                related_urls.append("MTI")
            if paragraph.find_all("a") is not None:
                for link in paragraph.find_all("a"):
                    try:
                        if "http" in link.get('href'):
                            related_urls.append(link.get('href'))
                        else:
                            related_urls.append("https:" + link.get('href'))
                    except:
                        print(paragraph.find_all("a"))
            magyar_hirlap[link] = {
                "related_links" : related_urls,
                "publish_date" : datetime}
    else:
        print(link, response.status_code)

https://www.magyarhirlap.hu/velemeny/20190331-a-lanchid 404
https://www.magyarhirlap.hu/velemeny/A_fejedelem 404
https://www.magyarhirlap.hu/velemeny/Tudatlansag 404
https://www.magyarhirlap.hu/velemeny/Penz_es_hatalom 404
https://www.magyarhirlap.hu/velemeny/Nemzetepito_felelosseg 404
https://www.magyarhirlap.hu/velemeny/Lelkiismeret-1 404
https://www.magyarhirlap.hu/velemeny/Ez_is_az_is-1 404
https://www.magyarhirlap.hu/velemeny/Mondak 404
https://www.magyarhirlap.hu/velemeny/Peldabeszedek 404
https://www.magyarhirlap.hu/velemeny/Ertek 404
[<a name="_GoBack"></a>, <a href="https://www.facebook.com/events/543168439531094/">https://www.facebook.com/events/543168439531094/</a>]
https://www.magyarhirlap.hu/velemeny/Megerzesek 404
https://www.magyarhirlap.hu/velemeny/Zene_es_politika 404
https://www.magyarhirlap.hu/velemeny/Tuzvesz 404
https://www.magyarhirlap.hu/velemeny/Felelem 404
https://www.magyarhirlap.hu/velemeny/A_bor 404


In [31]:
with open('magyar_hirlap.p', 'wb') as f:
    pickle.dump(magyar_hirlap, f)

In [32]:
magyar_hirlap

{'https://www.magyarhirlap.hu/kulfold/20191231-trump-szerint-iran-a-felelos-a-nagykovetseg-elleni-tamadasert': {'related_links': ['MTI'],
  'publish_date': '2019. december 31. kedd. 22:13'},
 'https://www.magyarhirlap.hu/sport/20191231-elegedett-a-mob-elnok-az-elmult-evvel': {'related_links': ['MTI'],
  'publish_date': '2019. december 31. kedd. 19:34'},
 'https://www.magyarhirlap.hu/kulfold/20191231-trump-elegedett-washington-es-moszkva-eyuttmukodesevel': {'related_links': ['MTI'],
  'publish_date': '2019. december 31. kedd. 18:47'},
 'https://www.magyarhirlap.hu/sport/20191231-milak-vilagcsucsatol-michelisz-tortenelmi-diadalaig-2019-legnagyobb-magyar-sportsikerei': {'related_links': ['MTI'],
  'publish_date': '2019. december 31. kedd. 18:33'},
 'https://www.magyarhirlap.hu/belfold/20191231-a-fidesz-tavozasa-a-neppart-szamara-a-veg-kezdete-lenne': {'related_links': ['MTI'],
  'publish_date': '2019. december 31. kedd. 17:31'},
 'https://www.magyarhirlap.hu/sport/20191231-szukult-a-vizil